# Welcome to `NeuroEvoBench` 👋 [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neuroevobench/neuroevobench/blob/main/examples/neb_introduction.ipynb)

In the following mini-tutorial we walk through the core functionality of the `NeuroEvoBench` benchmark. More specifically, we show how to run a random search sweep using a Simple Evolution Strategy (ES) implemented in [`evosax`](https://github.com/RobertTLange/evosax/) on a set of simple (and fast to evaluate) BBOB functions. All of the task evaluations follow the same protocol. Let's start by importing the benchmark and `evosax`.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

try:
    import evosax
    import neuroevobench
except Exception:
    !pip install -q git+https://github.com/RobertTLange/evosax.git@main
    !pip install -q git+https://github.com/neuroevobench/neuroevobench.git@main
    import evosax
    import neuroevobench

envpool not installed, Atari problems will not work.
envpool not installed, Atari problems will not work.
You need to install brax for Brax tasks:
  pip install git+https://github.com/google/brax.git@main


# Evolution Strategy Setup & `evosax` API 

In [2]:
import jax
import jax.numpy as jnp
from evosax import SimpleES

# Instantiate the evolution strategy instance
strategy = SimpleES(num_dims=2, popsize=10)

# Get default hyperparameters (e.g. lrate, etc.)
es_params = strategy.default_params
es_params = es_params.replace(init_min= -3, init_max=3)

# Initialize the strategy
rng = jax.random.PRNGKey(0)
state = strategy.initialize(rng, es_params)

# Have a look at the hyperparameters (change if desired)
es_params

EvoParams(c_sigma=0.1, c_m=1.0, sigma_init=1.0, init_min=-3, init_max=3, clip_min=-3.4028235e+38, clip_max=3.4028235e+38)

# `BBOB` task setup & running a single eval

In [3]:
from neuroevobench.problems.bbob import BBOBEvaluator

popsize = 10
num_dims = 2
es_config = {}
es_params = {}
num_eval_runs = 5
seed_id = 42
num_generations = 50


# 1. Setup task evaluator with strategy
evaluator = BBOBEvaluator(
    popsize=popsize,
    num_dims=num_dims,
    es_strategy=SimpleES,
    es_config=es_config,
    es_params=es_params,
    num_eval_runs=num_eval_runs,
    seed_id=seed_id,
)

# 2. Run the ES loop with logging
evaluator.run(num_generations)

START EVOLVING 2 PARAMETERS.
{'num_gens': 50} {'Sphere_mean': DeviceArray(0.00208277, dtype=float32), 'RosenbrockRotated_mean': DeviceArray(0.59280723, dtype=float32), 'Discus_mean': DeviceArray(3785.8782, dtype=float32), 'RastriginRotated_mean': DeviceArray(23.384197, dtype=float32), 'Schwefel_mean': DeviceArray(3.7671869, dtype=float32), 'BuecheRastrigin_mean': DeviceArray(28.045166, dtype=float32), 'AttractiveSector_mean': DeviceArray(0.550129, dtype=float32), 'Weierstrass_mean': DeviceArray(29.481977, dtype=float32), 'SchaffersF7_mean': DeviceArray(0.54004943, dtype=float32), 'GriewankRosenbrock_mean': DeviceArray(14.863263, dtype=float32), 'EllipsoidalOriginal_mean': DeviceArray(85.39087, dtype=float32), 'RastriginOriginal_mean': DeviceArray(23.81484, dtype=float32), 'LinearSlope_mean': DeviceArray(-266.49292, dtype=float32), 'StepEllipsoidal_mean': DeviceArray(1.4745407, dtype=float32), 'RosenbrockOriginal_mean': DeviceArray(1.094798, dtype=float32), 'EllipsoidalRotated_mean': De

In [4]:
# Average task performance across all BBOB functions
evaluator.fitness_eval

213.34922790527344

# Sequential Random Search Sweep

In [5]:
import copy
from mle_hyperopt import RandomSearch
from neuroevobench.problems import neb_eval_loops
from neuroevobench.hyperparams import HyperSpace

In [6]:
# Setup the strategy search space for sequential evaluation
hyperspace = HyperSpace(
    space={"real":
        {"sigma_init": {"begin": 0.01, "end": 0.5, "prior": "uniform"}}
    }
)

hyper_strategy = RandomSearch(
    **hyperspace.space,
    search_config={
        "refine_after": 40,
        "refine_top_k": 10
    },
    maximize_objective=True,
    seed_id=0,
    verbose=True,
)


from dotmap import DotMap
config = DotMap({
    "strategy_name": "SimpleES",
    "popsize": 10,
    "num_dims": 2,
    "es_config": {},
    "es_params": {},
    "num_eval_runs": 5,
    "seed_id": 42,
    "num_generations": 50
})

              MLE-Hyperopt Random Hyperspace 🚀                        
                                                                                
           🌻 Variable   Type   Search Range ↔                                  
          ────────────────────────────────────────────────────────────          
           sigma_init    real   Begin: 0.01, End: 0.5, Prior: uniform 

In [7]:
# Run the random search hyperparameter optimization loop
for search_iter in range(50):
    # Augment the default params with the proposed parameters
    proposal_params = hyper_strategy.ask()
    eval_config = copy.deepcopy(config)
    for k, v in proposal_params.items():
        eval_config.es_config[k] = v

    # Evaluate the parameter config by running a ES loop
    performance, solution = neb_eval_loops["bbob"](
        eval_config,
        log=None,
        search_iter=search_iter,
    )

    # Update search strategy - Note we minimize!
    hyper_strategy.tell(proposal_params, float(performance))

START EVOLVING 2 PARAMETERS.
{'num_gens': 50, 'iter_id': 0} {'Sphere_mean': DeviceArray(0.00016203, dtype=float32), 'RosenbrockRotated_mean': DeviceArray(0.27455124, dtype=float32), 'Discus_mean': DeviceArray(262.93015, dtype=float32), 'RastriginRotated_mean': DeviceArray(11.297349, dtype=float32), 'Schwefel_mean': DeviceArray(3.3801732, dtype=float32), 'BuecheRastrigin_mean': DeviceArray(2.0099385, dtype=float32), 'AttractiveSector_mean': DeviceArray(0.07498827, dtype=float32), 'Weierstrass_mean': DeviceArray(0.63930243, dtype=float32), 'SchaffersF7_mean': DeviceArray(0.17726558, dtype=float32), 'GriewankRosenbrock_mean': DeviceArray(8.454251, dtype=float32), 'EllipsoidalOriginal_mean': DeviceArray(20.129179, dtype=float32), 'RastriginOriginal_mean': DeviceArray(12.772013, dtype=float32), 'LinearSlope_mean': DeviceArray(-21.587547, dtype=float32), 'StepEllipsoidal_mean': DeviceArray(3.9564817, dtype=float32), 'RosenbrockOriginal_mean': DeviceArray(0.29550186, dtype=float32), 'Ellipsoi

┏━━━━━━━━━━━━━━━┳━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓    
   ┃ 📥 Total: 1   ┃ ID ┃ Obj. 📉 ┃ Configuration 🔖 - 06/12/2023 14:41:40 ┃    
   ┡━━━━━━━━━━━━━━━╇━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩    
   │ Best Overall  │ 0  │ 21.575  │ 'sigma_init': 0.279                    │    
   │ Best in Batch │ 0  │ 21.575  │ 'sigma_init': 0.279                    │    
   └───────────────┴────┴─────────┴────────────────────────────────────────┘

START EVOLVING 2 PARAMETERS.
{'num_gens': 50, 'iter_id': 1} {'Sphere_mean': DeviceArray(0.00027059, dtype=float32), 'RosenbrockRotated_mean': DeviceArray(0.3538096, dtype=float32), 'Discus_mean': DeviceArray(594.74286, dtype=float32), 'RastriginRotated_mean': DeviceArray(17.108307, dtype=float32), 'Schwefel_mean': DeviceArray(2.83757, dtype=float32), 'BuecheRastrigin_mean': DeviceArray(2.1512241, dtype=float32), 'AttractiveSector_mean': DeviceArray(0.11554252, dtype=float32), 'Weierstrass_mean': DeviceArray(1.4634793, dtype=float32), 'SchaffersF7_mean': DeviceArray(0.14000084, dtype=float32), 'GriewankRosenbrock_mean': DeviceArray(8.836069, dtype=float32), 'EllipsoidalOriginal_mean': DeviceArray(18.11529, dtype=float32), 'RastriginOriginal_mean': DeviceArray(12.940608, dtype=float32), 'LinearSlope_mean': DeviceArray(-49.27611, dtype=float32), 'StepEllipsoidal_mean': DeviceArray(1.3231727, dtype=float32), 'RosenbrockOriginal_mean': DeviceArray(0.384131, dtype=float32), 'EllipsoidalRotat

┏━━━━━━━━━━━━━━━┳━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓    
   ┃ 📥 Total: 2   ┃ ID ┃ Obj. 📉 ┃ Configuration 🔖 - 06/12/2023 14:42:08 ┃    
   ┡━━━━━━━━━━━━━━━╇━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩    
   │ Best Overall  │ 1  │ 34.405  │ 'sigma_init': 0.36                     │    
   │ Best in Batch │ 1  │ 34.405  │ 'sigma_init': 0.36                     │    
   └───────────────┴────┴─────────┴────────────────────────────────────────┘

START EVOLVING 2 PARAMETERS.
{'num_gens': 50, 'iter_id': 2} {'Sphere_mean': DeviceArray(0.0001942, dtype=float32), 'RosenbrockRotated_mean': DeviceArray(0.31130585, dtype=float32), 'Discus_mean': DeviceArray(358.07855, dtype=float32), 'RastriginRotated_mean': DeviceArray(12.835216, dtype=float32), 'Schwefel_mean': DeviceArray(2.9397182, dtype=float32), 'BuecheRastrigin_mean': DeviceArray(-6.807995, dtype=float32), 'AttractiveSector_mean': DeviceArray(0.07508777, dtype=float32), 'Weierstrass_mean': DeviceArray(0.54412144, dtype=float32), 'SchaffersF7_mean': DeviceArray(0.2482125, dtype=float32), 'GriewankRosenbrock_mean': DeviceArray(7.413588, dtype=float32), 'EllipsoidalOriginal_mean': DeviceArray(15.42706, dtype=float32), 'RastriginOriginal_mean': DeviceArray(10.985679, dtype=float32), 'LinearSlope_mean': DeviceArray(-30.565994, dtype=float32), 'StepEllipsoidal_mean': DeviceArray(1.5632765, dtype=float32), 'RosenbrockOriginal_mean': DeviceArray(0.32640016, dtype=float32), 'Ellipsoidal

...

┏━━━━━━━━━━━━━━━┳━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓    
   ┃ 📥 Total: 48  ┃ ID ┃ Obj. 📉 ┃ Configuration 🔖 - 06/12/2023 15:02:56 ┃    
   ┡━━━━━━━━━━━━━━━╇━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩    
   │ Best Overall  │ 34 │ 65.207  │ 'sigma_init': 0.019                    │    
   │ Best in Batch │ 47 │ 3.104   │ 'sigma_init': 0.08                     │    
   └───────────────┴────┴─────────┴────────────────────────────────────────┘

START EVOLVING 2 PARAMETERS.
{'num_gens': 50, 'iter_id': 48} {'Sphere_mean': DeviceArray(5.84814e-05, dtype=float32), 'RosenbrockRotated_mean': DeviceArray(0.20993891, dtype=float32), 'Discus_mean': DeviceArray(79.48189, dtype=float32), 'RastriginRotated_mean': DeviceArray(0.787512, dtype=float32), 'Schwefel_mean': DeviceArray(2.7975574, dtype=float32), 'BuecheRastrigin_mean': DeviceArray(-15.264173, dtype=float32), 'AttractiveSector_mean': DeviceArray(0.03827344, dtype=float32), 'Weierstrass_mean': DeviceArray(0.15553674, dtype=float32), 'SchaffersF7_mean': DeviceArray(0.04431811, dtype=float32), 'GriewankRosenbrock_mean': DeviceArray(9.647492, dtype=float32), 'EllipsoidalOriginal_mean': DeviceArray(3.1116703, dtype=float32), 'RastriginOriginal_mean': DeviceArray(1.6610087, dtype=float32), 'LinearSlope_mean': DeviceArray(16.231592, dtype=float32), 'StepEllipsoidal_mean': DeviceArray(1.5006441, dtype=float32), 'RosenbrockOriginal_mean': DeviceArray(0.07326716, dtype=float32), 'Ellipsoi

┏━━━━━━━━━━━━━━━┳━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓    
   ┃ 📥 Total: 49  ┃ ID ┃ Obj. 📉 ┃ Configuration 🔖 - 06/12/2023 15:03:23 ┃    
   ┡━━━━━━━━━━━━━━━╇━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩    
   │ Best Overall  │ 34 │ 65.207  │ 'sigma_init': 0.019                    │    
   │ Best in Batch │ 48 │ 7.289   │ 'sigma_init': 0.168                    │    
   └───────────────┴────┴─────────┴────────────────────────────────────────┘

START EVOLVING 2 PARAMETERS.
{'num_gens': 50, 'iter_id': 49} {'Sphere_mean': DeviceArray(7.543193e-05, dtype=float32), 'RosenbrockRotated_mean': DeviceArray(0.2178572, dtype=float32), 'Discus_mean': DeviceArray(65.843895, dtype=float32), 'RastriginRotated_mean': DeviceArray(0.4318529, dtype=float32), 'Schwefel_mean': DeviceArray(2.319941, dtype=float32), 'BuecheRastrigin_mean': DeviceArray(-7.2578063, dtype=float32), 'AttractiveSector_mean': DeviceArray(0.03047536, dtype=float32), 'Weierstrass_mean': DeviceArray(0.27195075, dtype=float32), 'SchaffersF7_mean': DeviceArray(0.10407119, dtype=float32), 'GriewankRosenbrock_mean': DeviceArray(8.873553, dtype=float32), 'EllipsoidalOriginal_mean': DeviceArray(5.414177, dtype=float32), 'RastriginOriginal_mean': DeviceArray(4.525599, dtype=float32), 'LinearSlope_mean': DeviceArray(8.507909, dtype=float32), 'StepEllipsoidal_mean': DeviceArray(1.6520121, dtype=float32), 'RosenbrockOriginal_mean': DeviceArray(0.06131207, dtype=float32), 'Ellipsoida

┏━━━━━━━━━━━━━━━┳━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓    
   ┃ 📥 Total: 50  ┃ ID ┃ Obj. 📉 ┃ Configuration 🔖 - 06/12/2023 15:03:50 ┃    
   ┡━━━━━━━━━━━━━━━╇━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩    
   │ Best Overall  │ 34 │ 65.207  │ 'sigma_init': 0.019                    │    
   │ Best in Batch │ 49 │ 7.334   │ 'sigma_init': 0.19                     │    
   └───────────────┴────┴─────────┴────────────────────────────────────────┘